## Setup

In [ ]:
import os
import requests
import pandas as pd

from dotenv import load_dotenv
from sklearn import set_config
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, log_loss

load_dotenv()
set_config(transform_output = 'pandas')

FROST_ID = os.getenv("FROST_ID")

## Load data

## Preprocess data

## Split data

## Temporary

## Baseline

In [ ]:
# y_val er fasit, target_actual
y_val = val["target_actual"]

# Naiv prediksjon: bruk target_sched direkte
y_pred_baseline = val["target_sched"]

# For log loss og AUC trenger vi "sannsynligheter".
# Vi kan bruke target_sched som sannsynlighet (0 eller 1).
auc_baseline = roc_auc_score(y_val, y_pred_baseline)
eps = 1e-3  # lite justeringsledd
y_pred_proba = val["target_sched"].replace({0: eps, 1: 1-eps})

logloss_baseline = log_loss(y_val, y_pred_proba)

print("Baseline AUC:", auc_baseline)
print("Baseline Log Loss:", logloss_baseline)

Baseline AUC: 0.7924117445456063
Baseline Log Loss: 1.4334834480322787


## Model

## Weather

In [ ]:
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    'sources': 'SN18700,SN90450',
    'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)',
    'referencetime': '2010-04-01/2010-04-03',
}
r = requests.get(endpoint, parameters, auth=(FROST_ID,''))
json = r.json()

ConnectionError: HTTPSConnectionPool(host='frost.met.no', port=443): Max retries exceeded with url: /observations/v0.jsonld?sources=SN18700%2CSN90450&elements=mean%28air_temperature+P1D%29%2Csum%28precipitation_amount+P1D%29%2Cmean%28wind_speed+P1D%29&referencetime=2010-04-01%2F2010-04-03 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x73f7111e3d90>: Failed to resolve 'frost.met.no' ([Errno -2] Name or service not known)"))

In [ ]:
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])

Data retrieved from frost.met.no!


In [ ]:
df = pd.DataFrame()
for i in range(len(data)):
    row = pd.DataFrame(data[i]['observations'])
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']
    df = pd.concat([df, row])

df = df.reset_index()

columns = ['sourceId','referenceTime','elementId','value','unit','timeOffset']
df2 = df[columns].copy()
# Convert the time value to something Python understands
df2['referenceTime'] = pd.to_datetime(df2['referenceTime'])

df2.head()

,sourceId,referenceTime,elementId,value,unit,timeOffset
0,SN18700:0,2010-04-01 00:00:00+00:00,mean(air_temperature P1D),3.2,degC,PT0H
1,SN18700:0,2010-04-01 00:00:00+00:00,mean(air_temperature P1D),3.0,degC,PT6H
2,SN18700:0,2010-04-01 00:00:00+00:00,sum(precipitation_amount P1D),13.5,mm,PT18H
3,SN18700:0,2010-04-01 00:00:00+00:00,sum(precipitation_amount P1D),29.0,mm,PT6H
4,SN18700:0,2010-04-01 00:00:00+00:00,mean(wind_speed P1D),1.7,m/s,PT0H
